## Recommendation System Based on Favorite Actor (Actor viewed most by User)

### Importing the Required Libraries & Files

In [1]:
import pandas as pd
import numpy as np
import sys

movies= pd.read_csv("movies.dat",encoding='latin-1',sep='\t')
movies.drop(['imdbID','spanishTitle','rtID','rtID', 'rtAllCriticsRating', 'rtAllCriticsNumReviews',\
       'rtAllCriticsNumFresh', 'rtAllCriticsNumRotten', 'rtAllCriticsScore',\
       'rtTopCriticsRating', 'rtTopCriticsNumReviews', 'rtTopCriticsNumFresh',\
       'rtTopCriticsNumRotten', 'rtTopCriticsScore',\
       'rtAudienceNumRatings', 'rtAudienceScore', 'rtPictureURL'],axis=1,inplace=True)
movies['title'].loc[0]='Toy Story'

movie_genre= pd.read_csv("movie_genres.dat",encoding='latin-1',sep='\t')
movie_genre = movie_genre.groupby('movieID').agg({ 'genre': ', '.join}).reset_index()

movie_director= pd.read_csv('movie_directors.dat',encoding='latin-1',sep='\t')

movie_director.drop(['directorID'],axis=1,inplace=True)

movie_actor= pd.read_csv('movie_actors.dat',encoding='latin-1',sep='\t')
movie_actor.drop(['actorID'],axis=1,inplace=True)

merge1= pd.merge(movies,movie_genre,left_on='id',right_on='movieID')
merge2= pd.merge(merge1,movie_director)
merge3= pd.merge(merge2,movie_actor)
merge3.drop(['movieID'],axis=1,inplace=True)

user_ratedmovies= pd.read_csv('user_ratedmovies.dat',encoding='latin-1',sep='\t')
user_ratedmovies.drop(['date_day', 'date_month', 'date_year',\
       'date_hour', 'date_minute', 'date_second'],axis=1,inplace=True)

final_data= pd.merge(merge3,user_ratedmovies,left_on='id',right_on='movieID')

C:\Users\Arpitha Ananth\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


### Identifying Favorite Actor of Every User

In [2]:
fav_actor1= pd.DataFrame(final_data.groupby(['userID','actorName'])['id'].count())
fav_actor1.reset_index(inplace=True)
fav_actor1= fav_actor1.merge(fav_actor1.loc[fav_actor1.groupby('userID').id.idxmax(), ['userID', 'actorName']])
fav_actor1.set_index(['userID'],inplace=True)

### Taking User ID as an input from User

In [3]:
user=int(input())

75


In [4]:
### Finding all the movies which has been watched by the User 
user_ratedmovies.groupby(['userID','movieID']).count()
subset_user_ratedmovies= user_ratedmovies[user_ratedmovies['userID']==user]

### Checking if User is present in the database or not
if len(subset_user_ratedmovies)>=1:
    print("User Found")
else:
    print("Invalid User ID, please re-check")

User Found


In [5]:
### Identifying the Favourite Actor of the User from User ID entered

actor_needed= fav_actor1[fav_actor1.index==user]['actorName']
for i in fav_actor1.index:
    if i==user:
        actor_needed=fav_actor1['actorName'][i]

### Finding all the Movies where the user's favorite actor has played the Lead Role
actor_mov= movie_actor[movie_actor['actorName']==actor_needed]

### Recommending Movies of Favourite Actor which has not been watched by User Yet
df_all = actor_mov.merge(subset_user_ratedmovies.drop_duplicates(), on=['movieID'],how='left', indicator=True)
recc1= df_all[df_all['_merge']=='left_only']
recc_movie= recc1[['movieID','actorName']]
final_recc= pd.merge(recc_movie,movies,left_on='movieID',right_on='id',how='left')
final_recc.drop(['id'],inplace=True,axis=1)

### List of Movies of Favourite Actor which has not been watched by User Yet
final_recc= final_recc.sort_values(by='rtAudienceRating',ascending=False)
final_recc.drop_duplicates('title',inplace=True,keep='first')

### Top 5 Movie Recommendations
final_recc.head(5)

,movieID,actorName,title,imdbPictureURL,year,rtAudienceRating
39,53972,Bruce Willis,Live Free or Die Hard,http://ia.media-imdb.com/images/M/MV5BNDQxMDE1...,2007,3.9
37,52281,Bruce Willis,Grindhouse,http://ia.media-imdb.com/images/M/MV5BMjA0MzEx...,2007,3.9
16,3068,Bruce Willis,The Verdict,http://ia.media-imdb.com/images/M/MV5BMTg1NzA5...,1982,3.8
0,18,Bruce Willis,Four Rooms,http://ia.media-imdb.com/images/M/MV5BMTcwOTMz...,1995,3.5
32,8984,Bruce Willis,Ocean's Twelve,http://ia.media-imdb.com/images/M/MV5BMTQzODQw...,2004,3.4
